In [1]:
%run utils.ipynb
random.seed(1)

In [2]:
root_dir = "./files/4_split_400/"

In [3]:
class_dirs = [os.path.join(root_dir, i) for i in os.listdir(root_dir)]
random.shuffle(class_dirs)
print(f"{len(class_dirs)} classes")

400 classes


In [4]:
# class_dirs = class_dirs[:300]

In [5]:
x_train, y_train = [], []
x_test, y_test = [], []

In [6]:
%%time
for class_label, class_dir in tqdm(enumerate(class_dirs)):
    train_list, test_list = list_split(get_all_images(class_dir, shuffle = False),
                                       ratio = [0.4, 0.6], shuffle = False)
#     try:
    for image_path in train_list:    
        _, _, roi = roi_extraction_with_hood(image_path)
        
#         _, _, roi = roi_extraction_1(image_path)
#         _, _, roi = roi_extraction_2(image_path)
        roi = resize(roi, is_gray = True, width = 256, height = 128)
#         feature = cvhog.compute(roi)
        feature = HOG_feature(roi)
        x_train.append(feature)
        y_train.append(class_label)

    for image_path in test_list:    
        _, _, roi = roi_extraction_with_hood(image_path)
#         _, _, roi = roi_extraction_1(image_path)
#         _, _, roi = roi_extraction_2(image_path)
        roi = resize(roi, is_gray = True, width = 256, height = 128)
#         feature = cvhog.compute(roi)
        feature = HOG_feature(roi)
        x_test.append(feature)
        y_test.append(class_label)
#     except:
#         print(image_path)


CPU times: user 36min 36s, sys: 6.54 s, total: 36min 43s
Wall time: 7min 8s


In [7]:
f_dim = x_test[0].shape[0]
print(f"feature dim : {f_dim}")

feature dim : 756


In [8]:
x_train = np.asarray(x_train).reshape(-1, f_dim)
x_test = np.asarray(x_test).reshape(-1, f_dim)

In [9]:
x_train.shape

# (4978, 756)

(4978, 756)

In [ ]:
%%time
# Grid Search
# Parameter Grid
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001, 0.00001, 10]}
 
# Make grid search classifier
clf_grid = GridSearchCV(svm.SVC(), param_grid, verbose=1, cv = 3, n_jobs = -1) #, pre_dispatch = cpu_count//2
 
# Train the classifier
clf_grid.fit(x_train, y_train)

# clf = grid.best_estimator_()
print("Best Parameters:\n", clf_grid.best_params_)
print("Best Estimators:\n", clf_grid.best_estimator_)

In [ ]:
# Make predictions on unseen test data
y_pred = clf_grid.predict(x_test)
# print("Accuracy: {}%".format(clf_grid.score(x_test, y_test) * 100 ))

In [ ]:
overall_acc = overall_accuracy(y_test, y_pred)
m_prec = mean_prec(y_test, y_pred)
m_recall = mean_recall(y_test, y_pred)

print(f"Over all acc : {overall_acc*100:.2f}%\nmean prec : {m_prec*100:.2f}%\nmean recall : {m_recall*100:.2f}%")

width = 256, height = 128
winSize = (256, 128) 
blockSize = (64, 64)
blockStride = (32, 32)
cellSize = (32,32)
nbins = 9

50

|metrics | roi 1 | roi 2 | roi ours |
| :-- :| :--:  | :--: | :--:|
| over all ac |92.83%|93.58%| 89.09%|
| mean prec | 92.85%| 92.84%| 89.30%|
|mean recall |90.77% |91.55% |85.47%|



100

|metrics | roi 1 | roi 2 | roi ours |
| :-- :| :--:  | :--: | :--:|
| over all ac |93.01%|93.58%| 89.09%|
| mean prec | 92.37%| 92.84%| 89.30%|
|mean recall |91.09% |91.55% |85.47%|



200

|metrics | roi 1 | roi 2 | roi ours |
| :-- :| :--:  | :--: | :--:|
| over all ac |90.69%|87.05%| 82.88%|
| mean prec | 90.13%| 86.60%| 82.28%|
|mean recall |87.65% |83.18% |77.95%|



300

|metrics | roi 1 | roi 2 | roi ours |
| :-- :| :--:  | :--: | :--:|
| over all ac |90.35%|87.05%| 82.69%|
| mean prec | 89.41%| 86.60%| 82.03%|
|mean recall |87.18%|83.18% | 77.46%|



400

|metrics | roi 1 | roi 2 | roi ours |
| :-- :| :--:  | :--: | :--:|
| over all ac | | | 81.14%|
| mean prec |  |  | 80.53%|
|mean recall | | | 75.57%|

